In [3]:
import pandas as pd
import numpy as np
import operator
from collections import defaultdict

In [4]:
df = pd.read_csv(open('../data/df2_million.csv'), low_memory=False)
df = df[df['text'].str.contains("00leave")== False]

df = df.dropna(axis=0)
df = df.drop('Unnamed: 0', 1)
print('有{}筆對話'.format(len(df)))

有999980筆對話


In [28]:
df = df[df['text'].str.contains("00add")== False]

In [29]:
df.count()

from    980613
to      980613
text    980613
room    980613
time    980613
dtype: int64

In [31]:
df = df.reset_index(drop=True)

In [32]:
df.head()

,from,to,text,room,time
0,962761420519280,1203470409728349,我明天快滿堂,30412dbe-bec7-4158-a607-91a22ef30e7f,1.474971e+12
1,1099645570121970,1098086130277625,真的,1f1b022f-3201-4e91-93d2-c29290f6de14,1.474971e+12
2,1200796473326885,824810620954251,哈哈哈哈哈哈哈哈哈哈,7c270335-4250-40dd-9c09-a50f641aa84a,1.474971e+12
3,1266237863406753,1119017804854401,一樣喔,6d6f6da5-f2f0-421e-8c12-e09088736373,1.474971e+12
4,1185976538142459,1187173851355534,台北哪阿,1c06456d-ff4c-4fda-8417-6e667cd50b92,1.474971e+12


## Groupby by 'from' column 

In [33]:
grouped_data = df.groupby('from')['text'].apply(list).reset_index()
print('have {} users'.format(len(grouped_data)))

have 5402 users


In [9]:
# df.loc[df['from'] == 'ud4097fbb4471c8c54512814bf0182e03']

In [34]:
grouped_data.head()
# type(grouped_data.text[0])

,from,text
0,1000309370066644,"[嘿, 哈哈, 爽賺一天假, 你沒放哦, 額真假, 為什麼, 哈哈哈哈哈, 不一樣啦, 我才..."
1,1000515923404164,"[滑fb 😂, 對啊哈哈哈, 嗨, ？, 你是女生？, 哈哈哈, 我終於遇到一個男的了, 幾..."
2,1001006620012161,"[挖 比我還慘阿..., 你還在讀書嗎, 快拿膠水去把它黏起來哈, 高中..., 你大幾阿,..."
3,1001093176683887,"[嗚啊, 哩賀, 哈哈, 嗨, 怎麼說, 星座男, 哈哈, 怎樣星座男, 那我不就是射手男,..."
4,1003346503122174,"[哈哈哈~, 真好 一個颱風假 我也想休息 ，可以我從早忙到晚呀, 作推甄資料呀, 哈囉,..."


In [35]:
print('id為 {} 的使用者傳送的訊息 ：{}'.format(grouped_data.iloc[0][0],grouped_data.iloc[0][1]))

id為 1000309370066644 的使用者傳送的訊息 ：['嘿', '哈哈', '爽賺一天假', '你沒放哦', '額真假', '為什麼', '哈哈哈哈哈', '不一樣啦', '我才剛來', '哇超猛', '覺得默契', '耶', '明天要上課覺得難過', '我去刷個牙', '好了', '嘿']


## set stop word

In [12]:
import jieba
import jieba.posseg as pseg
import jieba.analyse

stopwordset = set()
with open('jieba_txt/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
chars = set(' abcdefghijklmnopqrstuvwxyz0123456789()?@？!$,~:.。，-_&=/􀆿\r\n')
for c in chars:
    stopwordset.add(c)

In [13]:
# stopwordset

## 載入分詞檔做訓練

In [15]:
from gensim import models
import gensim
sentences = models.word2vec.LineSentence("../data/corpusSegDone.txt")
model = models.word2vec.Word2Vec(sentences, size=1, window=5, min_count=5, workers=1)

In [36]:
length = len(grouped_data)
length

5402

In [37]:
top10_vector = np.zeros(shape=(length,10))
jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
jieba.load_userdict("jieba_txt/userdict.txt")
for i in range(length):
    frequency = defaultdict(int)
    for j in range(len(grouped_data.iloc[i][1])):
        seglist = jieba.cut((grouped_data.iloc[i][1])[j])
        texts = [word for word in seglist if word not in stopwordset]
        for text in texts:
            frequency[text] += 1
            
    a = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)
    row = np.zeros(shape=(1,10))
    r = 10 if len(a) >= 10 else len(a)
    
    for x in range(r):
        if a[x][0] in model:
            show = float(model[a[x][0]])
            row[0][x] = show
    top10_vector[i] = row

Building prefix dict from /home/tp6han/gp_new/jieba_txt/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u0ddb98113e9461fac3a7658d9e24e45e.cache
Loading model cost 1.654 seconds.
Prefix dict has been built succesfully.


In [38]:
top10_vector.shape

(5402, 10)

In [39]:
model.corpus_count

11144516

# k-means clustering

In [40]:
from sklearn import cluster, datasets

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 3,random_state=1170).fit(top10_vector)

# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果 前10：")
print(cluster_labels[:10])
print("---")

分群結果 前10：
[0 0 0 0 0 0 0 0 0 0]
---


In [41]:
len(cluster_labels)

5402

In [42]:
unique, counts = np.unique(cluster_labels, return_counts=True)
dict(zip(unique, counts))

{0: 4769, 1: 274, 2: 359}

In [43]:
cluster_done = grouped_data
cluster_done['category'] = cluster_labels

In [44]:
cluster_done = cluster_done.sort_values(by='category')
# t = t.drop('from',1)

In [45]:
cluster_done = cluster_done.reset_index(drop=True)
cluster_done.head()

,from,text,category
0,1000309370066644,"[嘿, 哈哈, 爽賺一天假, 你沒放哦, 額真假, 為什麼, 哈哈哈哈哈, 不一樣啦, 我才...",0
1,273860868,"[你好, 你好啊, 你在幹嘛呀0..0, 我都躲在家了, 香港人哦餓, 哈哈, 颱風天好無聊...",0
2,273826628,"[開發者速速現, 沒關係, 聽說開發者是帥哥, 寶傑你怎麼看, 哈哈阿哈哈, 我是女生, 寶...",0
3,273624498,"[嗨, 高雄, 沒欸 你有喔, ......, 啊你哪裡, 還好?, 欸哈哈哈哈, 我要跟你...",0
4,273025952,"[摸我自己不行喔 我有你沒有的哈哈哈, 知 男 而 退, 我曾經夢到我被警察強吻耶😂...",0


In [46]:
cluster_done.count()
# small_clustered = cluster_done[6000:6300].reset_index(drop=True)

from        5402
text        5402
category    5402
dtype: int64

In [47]:
cluster_done.to_csv('../data/k_means_result2.csv',encoding='utf-8-sig',index=False)

## make dictionary = { from : category }

In [24]:
from_category = {}
for i, row in cluster_done.iterrows():
    from_category[row['from']] = row['category']

In [25]:
len(from_category)

12691

## groupby room 

In [27]:
df2 = df.copy()
df2 = df2.drop(['time','to'], axis=1)

In [28]:
df2['category'] = df2['from'].map(from_category)

In [29]:
df2.head()
df2 = df2.dropna(axis=0)
df2.count()


from        999961
text        999961
room        999961
category    999961
dtype: int64

In [30]:
df2.head()

,from,text,room,category
0,ud4097fbb4471c8c54512814bf0182e03,wd,8aee77a7-1d90-4d0f-b3c3-3f7b94a90e36,1
1,ud4097fbb4471c8c54512814bf0182e03,嗨嗨,fe214cfc-22d1-4736-92e7-d133190142b8,1
2,u96e826dccd3c9db56d684a5ddffc3154,蛤？,fe214cfc-22d1-4736-92e7-d133190142b8,1
3,ud4097fbb4471c8c54512814bf0182e03,你是第二個使用者ＱＱ,fe214cfc-22d1-4736-92e7-d133190142b8,1
4,u96e826dccd3c9db56d684a5ddffc3154,哈哈哈哈哈哈哈,fe214cfc-22d1-4736-92e7-d133190142b8,1


In [84]:
p1 = df2.groupby('room')['from'].aggregate(lambda x: set(x)).reset_index()
p2 = df2.groupby('room')['category'].aggregate(lambda x: set(x)).reset_index()
c = df2.groupby('room')['text'].agg(['count']).reset_index()
t = df2.groupby('room')['text'].apply(list).reset_index()

In [32]:
print('have {} {} rooms'.format(len(p1),len(p2)))

have 39423 39423 rooms


## new df  group by room

In [85]:
room_grouped = pd.concat([p1,p2['category'],c['count'],t['text']],axis=1)

In [86]:
room_grouped.columns

Index(['room', 'from', 'category', 'count', 'text'], dtype='object')

In [87]:
room_grouped.head()

,room,from,category,count,text
0,0000d3d1-cd74-4c6a-8ab6-8c9c0df594aa,"{1313316695368083, u84ff3587ce973343ca6328d688...",{2},2,"[嗨嗨, 请问有人吗]"
1,000129e0-e9c9-4e51-8b7a-b782d220894e,{1127853807261849},{1},1,[安]
2,0002fc8b-0951-47b0-9348-bf7decae9d93,"{955305891265054, 966587086804396}","{1, 2}",6,"[嗨, 你好喔, 你好喔~, 男生, 你呢, me too]"
3,000418d0-779f-469a-ad5b-7fcfd6da4959,"{ua725d2fbddf188fb2e97f67bbd36d056, 1084323831...",{1},26,"[你好, 嗨, 嗨, 男, 女, 住哪呀?, 高雄, 台中, 那多大, 18\n你呢, 21..."
4,00054188-59bc-49cf-aea2-e8ddd7057849,"{1192735997454971, 1157361747619580}",{1},162,"[妳好, 你好啊阿, 真假的, 這真的很酷, 假的, 迪卡上看到的嗎, 對啊, 你是我第一個..."


## delete the row (room) which has only one user 

In [88]:
for index, row in room_grouped.iterrows():
    l = len(row['from'])
    if l == 1 :
        room_grouped.drop(index, inplace=True)

In [89]:
room_grouped.count()

room        30535
from        30535
category    30535
count       30535
text        30535
dtype: int64

In [90]:
room_grouped.head()

,room,from,category,count,text
0,0000d3d1-cd74-4c6a-8ab6-8c9c0df594aa,"{1313316695368083, u84ff3587ce973343ca6328d688...",{2},2,"[嗨嗨, 请问有人吗]"
2,0002fc8b-0951-47b0-9348-bf7decae9d93,"{955305891265054, 966587086804396}","{1, 2}",6,"[嗨, 你好喔, 你好喔~, 男生, 你呢, me too]"
3,000418d0-779f-469a-ad5b-7fcfd6da4959,"{ua725d2fbddf188fb2e97f67bbd36d056, 1084323831...",{1},26,"[你好, 嗨, 嗨, 男, 女, 住哪呀?, 高雄, 台中, 那多大, 18\n你呢, 21..."
4,00054188-59bc-49cf-aea2-e8ddd7057849,"{1192735997454971, 1157361747619580}",{1},162,"[妳好, 你好啊阿, 真假的, 這真的很酷, 假的, 迪卡上看到的嗎, 對啊, 你是我第一個..."
5,0005ce30-1ab7-4b4f-8861-684a88e4f94b,"{1020089304775462, 954276064700164}",{1},175,"[我不給ㄩ, 我也沒ㄩ你啊, 到處都是什麼霸主的, 那是什麼, 晚上一直聽到, 不知道, 感..."


### 每間聊天室平均聊幾句

In [98]:
room_grouped['count'].mean()

32.296905190764697

# 證明分群有意義

## 1. 同一群的使用者平均聊天句數比不同群的使用者平均聊天句數還多

In [117]:
same_cat_count = 0
all_text = 0
other_user = 0
other_all_text = 0
for index, row in room_grouped.iterrows():
    l = len(row['category'])
    t_len = len(row['text'])
# same cluster
    if l == 1:
        same_cat_count+=1
        all_text+=t_len
# different cluster
    else:
        other_user+=1
        other_all_text+=t_len

In [118]:
print('同一群的使用者有 {} 組, 平均聊天句數 : {}'.format(same_cat_count,all_text/same_cat_count))
print('不同群的使用者有 {} 組, 平均聊天句數 : {}'.format(other_user ,other_all_text/other_user))

同一群的使用者有 18187 組, 平均聊天句數 : 41.353164348160774
不同群的使用者有 12348 組, 平均聊天句數 : 18.95821185617104


## 2. 聊天句數超過 1000 句的使用者，同群的有幾組

In [123]:
gt1000 = room_grouped[room_grouped['count']>1000]

In [124]:
gt1000_same_cat = 0
for i, row in temp.iterrows():
    l = len(row['category'])
    if l == 1:
        gt1000_same_cat+=1

In [125]:
print('共有 {} 組，同類的有 {} 組'.format(len(gt1000),gt1000_same_cat))

共有 29 組，同類的有 26 組


In [105]:
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
t.to_csv('../data/k_means_result.csv',encoding='utf-8-sig')